In [ ]:
##Importing the Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
train_data = pd.read_csv('/content/train_u6lujuX_CVtuZ9i.csv')
test_data = pd.read_csv('/content/test_Y3wMUE5_7gLdaTN.csv')
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
##Data Visualization and Analysis
###Train Dataset
train_df
train_df.info()
train_df.describe()
###Univariate Analysis
# 2. Frequency Distribution for Categorical Variables
categorical_vars = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area', 'Loan_Status']
frequency_distributions = {var: train_df[var].value_counts() for var in categorical_vars}
####Bar Charts
# Create a frequency table for the 'Gender' variable
gender_counts = train_df['Gender'].value_counts()

# Create a bar chart of the gender counts
plt.bar(gender_counts.index, gender_counts.values)

# Set the chart title and labels
plt.title('Gender Distribution of Loan Borrowers')
plt.xlabel('Gender')
plt.ylabel('Number of Borrowers')

# Show the chart
plt.show()
####Histograms
numerical_vars = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
for var in numerical_vars:
    plt.hist(train_df[var], bins=20)
    plt.title(f'Histogram of {var}')
    plt.xlabel(var)
    plt.ylabel('Frequency')
    plt.show()
####Box Plots
for var in numerical_vars:
    sns.boxplot(x=train_df[var])
    plt.title(f'Box Plot of {var}')
    plt.show()
####Pi-chart
for var in ['Gender', 'Education']:
    train_df[var].value_counts().plot.pie(autopct='%1.1f%%')
    plt.title(f'Pie Chart of {var}')
    plt.ylabel('')
    plt.show()
####Kernel Density Estimation (KDE)
for var in numerical_vars:
    sns.kdeplot(train_df[var], shade=True)
    plt.title(f'KDE Plot of {var}')
    plt.xlabel(var)
    plt.ylabel('Density')
    plt.show()
###Bivariate Analysis
####Correlation Matrix
correlation_matrix = train_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title('Correlation Matrix')
plt.show()
####Scatter Plot
plt.scatter(train_df['ApplicantIncome'], train_df['LoanAmount'])
plt.title('Scatter Plot: ApplicantIncome vs. LoanAmount')
plt.xlabel('ApplicantIncome')
plt.ylabel('LoanAmount')
plt.show()
####Stacked Bar Plots
crosstab = pd.crosstab(train_df['Credit_History'], train_df['Loan_Status'])
crosstab.plot(kind='bar', stacked=True)
plt.title('Stacked Bar Plot: Credit History vs. Loan Status')
plt.xlabel('Credit History')
plt.ylabel('Count')
plt.show()
####Pair Plots
sns.pairplot(train_df, hue='Loan_Status', diag_kind='kde')
plt.suptitle('Pairplot of Numerical Variables with Loan Status')
plt.show()
####Violin Plots
sns.violinplot(x='Loan_Status', y='ApplicantIncome', data=train_df)
plt.title('Violin Plot: Loan Status vs. Applicant Income')
plt.xlabel('Loan Status')
plt.ylabel('Applicant Income')
plt.show()
####Heat map for cross tabulation
crosstab = pd.crosstab(train_df['Education'], train_df['Loan_Status'])
sns.heatmap(crosstab, annot=True, cmap="YlGnBu")
plt.title('Heatmap: Education vs. Loan Status')
plt.show()
####Regression Plot
sns.regplot(x='ApplicantIncome', y='LoanAmount', data=train_df)
plt.title('Regression Plot: Applicant Income vs. Loan Amount')
plt.xlabel('Applicant Income')
plt.ylabel('Loan Amount')
plt.show()
###Multivariate Analysis
train_df.plot.line()
##Data pre-processing
###Check for NULL values and filling them
train_df.isnull().sum()
train_df['LoanAmount']=train_df['LoanAmount'].fillna(train_df['LoanAmount'].mean())
train_df['Loan_Amount_Term']=train_df['Loan_Amount_Term'].fillna(train_df['Loan_Amount_Term'].mean())
train_df['Credit_History']=train_df['Credit_History'].fillna(train_df['Credit_History'].mean())
train_df['Gender']=train_df['Gender'].fillna(train_df['Gender'].mode()[0])
train_df['Married']=train_df['Married'].fillna(train_df['Married'].mode()[0])
train_df['Dependents']=train_df['Dependents'].fillna(train_df['Dependents'].mode()[0])
train_df['Self_Employed']=train_df['Self_Employed'].fillna(train_df['Self_Employed'].mode()[0])
###Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df['Gender'] = label_encoder.fit_transform(train_df['Gender'])
train_df['Married'] = label_encoder.fit_transform(train_df['Married'])
train_df['Dependents'] = label_encoder.fit_transform(train_df['Dependents'])
train_df['Education'] = label_encoder.fit_transform(train_df['Education'])
train_df['Self_Employed'] = label_encoder.fit_transform(train_df['Self_Employed'])
train_df['Property_Area'] = label_encoder.fit_transform(train_df['Property_Area'])
train_df['Loan_Status'] = label_encoder.fit_transform(train_df['Loan_Status'])
###Feature Scaling (Standardization)
train_df=train_df.drop(columns=["Loan_ID"],axis=1)
x=train_df.iloc[:,:-1]
y=train_df.Loan_Status
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

x_scaled_df = pd.DataFrame(x_scaled, columns=x.columns)
x_scaled_df.head()
y
from imblearn.under_sampling import RandomUnderSampler

# Assuming you have X and y as your feature matrix and target variable
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(x_scaled, y)
X_resampled.shape,x.shape, y.shape, y_resampled.shape
ax=y_resampled.value_counts().plot.pie(autopct='%.2f')
_=ax.set_title("under-sampling")
###Splitting the Data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_resampled,y_resampled,test_size=0.3,random_state=10)
x_train.shape, x_test.shape, y_train.shape, y_test.shape
##Model Building
###Decision Tree Model
from sklearn.tree import DecisionTreeClassifier

model_1 = DecisionTreeClassifier(random_state=42)
model_1.fit(x_train, y_train)
model_1_predictions = model_1.predict(x_test)
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Decision Tree Model:")
print("Accuracy:", accuracy_score(y_test, model_1_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, model_1_predictions))
print("Classification Report:\n", classification_report(y_test, model_1_predictions))
####Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Decision Tree
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Perform Grid Search Cross Validation
grid_search_dt = GridSearchCV(dt_model, param_grid_dt, cv=5, scoring='accuracy')
grid_search_dt.fit(x_train, y_train)

# Get the best parameters
best_params_dt = grid_search_dt.best_params_

# Train the Decision Tree model with the best parameters
final_dt_model = DecisionTreeClassifier(**best_params_dt, random_state=42)
final_dt_model.fit(x_train, y_train)

# Make predictions on the test data
dt_predictions = final_dt_model.predict(x_test)

# Evaluate the model
print("Decision Tree Model:")
print("Best Parameters:", best_params_dt)
print("Accuracy:", accuracy_score(y_test, dt_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, dt_predictions))
print("Classification Report:\n", classification_report(y_test, dt_predictions))

####Validation
from sklearn.model_selection import cross_val_score

# Perform Cross-Validation
cv_scores_dt = cross_val_score(final_dt_model, x_train, y_train, cv=5, scoring='accuracy')

# Evaluate the model
print("Decision Tree Model:")
print("Best Parameters:", best_params_dt)
print("Cross-Validation Scores:", cv_scores_dt)
print("Mean CV Accuracy:", np.mean(cv_scores_dt))

###Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

model_2 = RandomForestClassifier(random_state=42)
model_2.fit(x_train, y_train)
model_2_predictions = model_2.predict(x_test)

# Calculate accuracy as a performance metric
print("Random Forest classifier:")
print("Accuracy:", accuracy_score(y_test, model_2_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, model_2_predictions))
print("Classification Report:\n", classification_report(y_test, model_2_predictions))
####Hyper-parameter tuning
# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Perform Grid Search Cross Validation
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='accuracy')
grid_search_rf.fit(x_train, y_train)

# Get the best parameters
best_params_rf = grid_search_rf.best_params_

# Train the Random Forest model with the best parameters
final_rf_model = RandomForestClassifier(**best_params_rf, random_state=42)
final_rf_model.fit(x_train, y_train)

# Make predictions on the test data
rf_predictions = final_rf_model.predict(x_test)

# Evaluate the model
print("Random Forest Model:")
print("Best Parameters:", best_params_rf)
print("Accuracy:", accuracy_score(y_test, rf_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_predictions))
print("Classification Report:\n", classification_report(y_test, rf_predictions))
####Validation
# Perform Cross-Validation
cv_scores_rf = cross_val_score(final_rf_model, x_train, y_train, cv=5, scoring='accuracy')

# Evaluate the model
print("Random Forest Model:")
print("Best Parameters:", best_params_rf)
print("Cross-Validation Scores:", cv_scores_rf)
print("Mean CV Accuracy:", np.mean(cv_scores_rf))

###KNN Classifier
from sklearn.neighbors import KNeighborsClassifier

model_3 = KNeighborsClassifier()
model_3.fit(x_train, y_train)
model_3_predictions = model_3.predict(x_test)

# Calculate accuracy as a performance metric
print("KNN:")
print("Accuracy:", accuracy_score(y_test, model_3_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, model_3_predictions))
print("Classification Report:\n", classification_report(y_test, model_3_predictions))

####Hyper-parameter tuning
# Define the parameter grid for KNN
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

# Create the KNN model
knn_model = KNeighborsClassifier()

# Perform Grid Search Cross Validation
grid_search_knn = GridSearchCV(knn_model, param_grid_knn, cv=5, scoring='accuracy')
grid_search_knn.fit(x_train, y_train)

# Get the best parameters
best_params_knn = grid_search_knn.best_params_

# Train the KNN model with the best parameters
final_knn_model = KNeighborsClassifier(**best_params_knn)
final_knn_model.fit(x_train, y_train)

# Make predictions on the test data
knn_predictions = final_knn_model.predict(x_test)

# Evaluate the model
print("KNN Model:")
print("Best Parameters:", best_params_knn)
print("Accuracy:", accuracy_score(y_test, knn_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, knn_predictions))
print("Classification Report:\n", classification_report(y_test, knn_predictions))
####Validation
# Perform Cross-Validation
cv_scores_knn = cross_val_score(final_knn_model, x_train, y_train, cv=5, scoring='accuracy')

# Evaluate the model
print("KNN Model:")
print("Best Parameters:", best_params_knn)
print("Cross-Validation Scores:", cv_scores_knn)
print("Mean CV Accuracy:", np.mean(cv_scores_knn))

###XG Boost classifier
from xgboost import XGBClassifier

model_4 = XGBClassifier(random_state=42)
model_4.fit(x_train, y_train)
model_4_predictions = model_4.predict(x_test)

# Calculate accuracy as a performance metric
print("XG Boost:")
print("Accuracy:", accuracy_score(y_test, model_4_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, model_4_predictions))
print("Classification Report:\n", classification_report(y_test, model_4_predictions))

####Hyper-parameter tuning
# Define the parameter grid for XGBoost
param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Create the XGBoost model
xgb_model = XGBClassifier(random_state=42)

# Perform Grid Search Cross Validation
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=5, scoring='accuracy')
grid_search_xgb.fit(x_train, y_train)

# Get the best parameters
best_params_xgb = grid_search_xgb.best_params_

# Train the XGBoost model with the best parameters
final_xgb_model = XGBClassifier(**best_params_xgb, random_state=42)
final_xgb_model.fit(x_train, y_train)

# Make predictions on the test data
xgb_predictions = final_xgb_model.predict(x_test)

# Evaluate the model
print("XGBoost Model:")
print("Best Parameters:", best_params_xgb)
print("Accuracy:", accuracy_score(y_test, xgb_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, xgb_predictions))
print("Classification Report:\n", classification_report(y_test, xgb_predictions))
####Validation
cv_scores_xgb = cross_val_score(final_xgb_model, x_train, y_train, cv=5, scoring='accuracy')

# Evaluate the model
print("XGBoost Model:")
print("Best Parameters:", best_params_xgb)
print("Cross-Validation Scores:", cv_scores_xgb)
print("Mean CV Accuracy:", np.mean(cv_scores_xgb))

###Ensemble Model
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
# Create the ensemble model using a VotingClassifier
ensemble_model = VotingClassifier(
    estimators=[
        ('DecisionTree', model_1),
        ('RandomForest', model_2),
        ('KNN', model_3),
        ('XGBoost', model_4)
    ],
    voting='hard'  # 'hard' for majority class voting
)

# Train the ensemble model on the undersampled training data
ensemble_model.fit(x_train, y_train)

# Make predictions on the test data
ensemble_predictions = ensemble_model.predict(x_test)

# Calculate accuracy as a performance metric for the ensemble model
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print("Ensemble Model Accuracy:", ensemble_accuracy)

print("Ensemble Model:")
print("Accuracy:", accuracy_score(y_test, ensemble_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, ensemble_predictions))
print("Classification Report:\n", classification_report(y_test, ensemble_predictions))
import joblib

# Save the best model to a file
joblib.dump(ensemble_model, 'best_model.pkl')

